In [2]:
import unicodedata

import joblib
import pandas as pd
from nltk.corpus import stopwords

def remove_nonlatin(string):
    new_chars = []
    for char in string:
        if char == '\n':
            new_chars.append(' ')
            continue
        try:
            if unicodedata.name(char).startswith(('LATIN', 'SPACE')):
                new_chars.append(char)
        except:
            continue
    return ''.join(new_chars)

def pre_processor(text):
    stops = set(stopwords.words("portuguese"))
    text = remove_nonlatin(text)
    words = text.lower().split()
    words = ' '.join([w for w in words if not w in stops])
    return words

def prepare_data(dataset):
    df = pd.read_csv(dataset, sep=';')
    df.descricao = df.descricao.str.replace('Produto Novo', '')
    df.dropna(inplace=True)
    df['nome_desc'] = df.nome + ' ' + df.descricao
    df.nome_desc = df.nome_desc.apply(pre_processor)
    df.drop(['nome','descricao'], axis=1, inplace=True)
    return df



In [3]:
df = pd.read_csv("https://s3.amazonaws.com/automl-example/produtos.csv", sep=";")

In [4]:
data = prepare_data("https://s3.amazonaws.com/automl-example/produtos.csv")

In [5]:
data.sample(5)

,categoria,nome_desc
169,livro,enfield real sobrenatural entalhes madeira é p...
2682,maquiagem,pinceis kabuki grande pincel maquiagem kit c u...
2095,maquiagem,cílios postiço fio fio seda b c d mix diversas...
474,livro,livro cemitério stephen king louis creed jovem...
2552,maquiagem,kit designer sobrancelha profissional universa...


In [6]:
import joblib

import autosklearn.classification as automl
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

def train(df, fit_file):
    print("Training...")
    train_size = 0.75
    vectorizer = CountVectorizer(
        analyzer="word",
        tokenizer=None,
        preprocessor=None,
        stop_words=None
    )
    clf = automl.AutoSklearnClassifier(
        include_preprocessors=["no_preprocessing",],
        exclude_preprocessors=None
    )
    encoder = LabelEncoder()

    y = df.categoria
    y = encoder.fit_transform(y)

    X = vectorizer.fit_transform(df.nome_desc)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=train_size
    )
    clf.fit(X_train, y_train)

    accuracy = clf.score(X_test, y_test)
    msg = "Accuracy with {:.0%} of testing data: {:.1%}".format(1 - train_size, accuracy)
    print(msg)
    joblib.dump(clf, fit_file)
    joblib.dump(encoder, 'encoder_%s' % fit_file)
    joblib.dump(vectorizer, 'vectorizer_%s' % fit_file)

In [7]:
train(data, "model")

Training...


/home/rafael/.virtualenvs/automl-example-engine-env/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Accuracy with 25% of testing data: 98.4%


In [8]:
def predict(text, fit_file='classifier.pkl'):
    clf = joblib.load(fit_file)
    vectorizer = joblib.load('vectorizer_%s' % fit_file)
    encoder = joblib.load('encoder_%s' % fit_file)

    text = pre_processor(text)
    text = vectorizer.transform([text])
    resp = clf.predict(text)[0]
    resp = encoder.classes_[resp]
    return resp

In [10]:
predict('The Witcher III Wild Hunt: Edição Completa', 'model')

'livro'

In [11]:
text = """Torne-se um profissional assassino de monstros e embarque em uma aventura de 
proporções épicas! Após o seu lançamento, The Witcher 3: Wild Hunt tornou-se um clássico 
instantâneo, reivindicando mais de 250 prêmios Game of the Year. Agora você pode desfrutar 
desta enorme aventura, com mais de 100 horas de duração, de mundo aberto, juntamente com suas 
expansões que valem mais de 50 horas adicionais na história de jogo. Esta edição inclui todo o 
conteúdo adicional - novas armas, armaduras, roupas de acompanhante, novo modo de jogo e missões paralelas."""

predict(text, 'model')

'game'

In [14]:
clf = joblib.load('model')
clf

AutoSklearnClassifier(delete_output_folder_after_terminate=True,
           delete_tmp_folder_after_terminate=True,
           disable_evaluator_output=False, ensemble_nbest=50,
           ensemble_size=50, exclude_estimators=None,
           exclude_preprocessors=None, get_smac_object_callback=None,
           include_estimators=None,
           include_preprocessors=['no_preprocessing'],
           initial_configurations_via_metalearning=25,
           ml_memory_limit=3072, output_folder=None,
           per_run_time_limit=360, resampling_strategy='holdout',
           resampling_strategy_arguments=None, seed=1, shared_mode=False,
           smac_scenario_args=None, time_left_for_this_task=3600,
           tmp_folder=None)

In [21]:
print(clf.get_models_with_weights())

[(0.66000000000000003, SimpleClassificationPipeline({'preprocessor:__choice__': 'no_preprocessing', 'classifier:libsvm_svc:gamma': 0.08351549479967445, 'balancing:strategy': 'weighting', 'categorical_encoding:__choice__': 'one_hot_encoding', 'classifier:__choice__': 'libsvm_svc', 'classifier:libsvm_svc:C': 1255.9137433589424, 'categorical_encoding:one_hot_encoding:use_minimum_fraction': 'True', 'classifier:libsvm_svc:tol': 0.00017919875199222518, 'classifier:libsvm_svc:shrinking': 'True', 'categorical_encoding:one_hot_encoding:minimum_fraction': 0.004980497345831963, 'imputation:strategy': 'mean', 'classifier:libsvm_svc:max_iter': -1, 'classifier:libsvm_svc:kernel': 'rbf', 'rescaling:__choice__': 'normalize'},
dataset_properties={
  'multiclass': True,
  'multilabel': False,
  'signed': False,
  'target_type': 'classification',
  'sparse': True,
  'task': 2})), (0.26000000000000001, SimpleClassificationPipeline({'preprocessor:__choice__': 'no_preprocessing', 'classifier:sgd:alpha': 2.0